In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-11 04:10:08--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-11 04:10:08 (41.3 MB/s) - ‘minsearch.py.1’ saved [4073/4073]



In [2]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import minsearch

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [4]:
import json

In [5]:
from openai import OpenAI

In [6]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [9]:
index.fit(documents)

In [10]:
client = OpenAI()

In [11]:
def search(query):
    boost = {'question': 3.0, 'section' : 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """

    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION:{question}
    CONTEXT:{context}
    """.strip()
    
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context= context).strip()
    return prompt

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[{"role":"user", "content":prompt}]
)
    return response.choices[0].message.content

In [14]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
rag(query)

'To run Kafka, follow the steps for executing Java Kafka components in the terminal: \n\nNavigate to your project directory and use the Java class path command:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nJust replace `<jar_name>` with the actual name of your JAR file.'

In [21]:
from elasticsearch import Elasticsearch

In [22]:
es_client = Elasticsearch('http://localhost:9200')

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
from tqdm.auto import tqdm

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████| 948/948 [00:03<00:00, 304.30it/s]


In [30]:
query = 'I just discovered the course, can I still join it?'

In [37]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [39]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer



In [40]:
rag(query)

"Yes, you can still join the course even after the start date. You are eligible to submit homeworks even if you haven't registered. However, please be mindful of the deadlines for turning in the final projects to avoid leaving everything for the last minute."